In [25]:
import pandas as pd
import jsonlines
import json
import os

In [40]:
all_files = os.listdir('./retrieval_data/candidate_pool_url/') ## candidate pool 경로
jsonl_file = [os.path.splitext(file)[0] for file in all_files if file.endswith('.jsonl')] ## jsonl인 확장자만 불러와서 파일 명만 추출

In [39]:
# candiate_pool로 규정 데이터를 학습하는 방법

def convert_cddpool_dialog(file_name):
    ## candidate_pool로 원본 데이터 불러오기 
    ## ori_text는 제목과 내용이 붙어있어서 분할하기 어려울 것 같음
    candidate_pool_path = f'./retrieval_data/candidate_pool_url/{file_name}.jsonl' ## candidate pool 있는 경로로 변환
    dialog_output_path = f'./sft_dialog_data/text_dialog/{file_name}.json'         ## dialog 경로로 변환

    candidate_pool = []
    with jsonlines.open(candidate_pool_path, 'r') as file:
        for pool in file:
            candidate_pool.append(pool)  # JSON 파일을 파싱
            
    sft_dialog = []
    for row in candidate_pool:
        sft_dialog.append([
            {"role": "system", "content" : "당신은 육군 스마트부대 상담사입니다. 다음 내용을 읽고 친절하게 답변할 수 있도록 합니다."}, ## 시스템 메세지 넣기
            {"role":"user", "content":f"{row['reference']}의 {row['url']}에 대해 그대로 읽어줘"}, 
            {"role":"assistant", "content":f"{row['reference']}의 {row['url']}에 대해 읽어드리겠습니다.\n {row['text']}"}
        ])

    with open(dialog_output_path, 'w', encoding='utf-8') as file:
        json.dump(sft_dialog, file, ensure_ascii=False)


# 모든 문서 적용시키기       
for file_name in jsonl_file:
    convert_cddpool_dialog(file_name)

In [69]:
all_files = os.listdir('.') ## qa데이터 정제 엑셀파일 폴더 경로
excel_file = [os.path.splitext(file)[0] for file in all_files if file.endswith('.xlsx')] ## xlsx인 확장자만 불러와서 파일 명만 추출

In [93]:
## 엑셀 파일 단위로 데이터 생성
def convert_rag_dialog(excel_file_name):
    ## 파일이 엑셀에 묶여있으니까 엑셀 파일 먼저
    qa_file_path = '' + excel_file_name + '.xlsx' ## qa 정제 set 데이터 경로
    
    ## 엑셀 파일 내 시트 이름 리스트
    xls = pd.ExcelFile(qa_file_path)
    sheet_name_list = xls.sheet_names
    
    for sheet_nm in sheet_name_list:
        
        ## dialog 데이터 셋 데이터 경로
        qa_output_path = f'./sft_dialog_data/rag_dialog/{sheet_nm}.json'
        ## 파일 불러오기
        qa_table = pd.read_excel(qa_file_path, sheet_name=sheet_nm)
        
        sft_dialog = []
        for index, row in qa_table.iterrows():
            idx = row['ori_text'].find(')') ## ) 기준으로 조항 이름과 내용 나누기
            doc_idx = row['ori_text'][:idx+1].strip() ## 조항 이름
            doc_cont = row['ori_text'][idx+1:].strip() ## 조항 본문
            
            ## user content 가 너무 길어서 따로 빼서 봄. 수정해도 됩니다.
            user_content = f"""주어진 질문에 차근차근 생각하고 질문에 대한 답변을 해주세요.
'제가 알고 있는 정보' 중 유효한 정보만을 선별하여, 내용 왜곡 없이 질문에 답변하세요. 
반드시 '제가 알고 있는 정보'만을 사용하여 답변하여야 합니다. 
질문에 대한 유효한 정보가 없다면 답변할 수 없다고 말해주세요.
#제가 알고 있는 정보1:{doc_cont}
#참조: {sheet_nm}의 {doc_idx}

#질문: {row['question']}

답변형식은 답변: [내용]을 유지하세요."""
            
            sft_dialog.append([
                {"role": "system", "content" : "당신은 육군 스마트부대 상담사입니다. 다음 내용을 읽고 친절하게 답변할 수 있도록 합니다."}, ## 시스템 메세지 넣기
                {"role":"user", "content": user_content}, ## 여기에 따로 넣어도 됨
                {"role":"assistant", "content": row["answer"]}
            ])
            
        with open(qa_output_path, 'w', encoding='utf-8') as file:
            json.dump(sft_dialog, file, ensure_ascii=False)

In [65]:
# 모든 문서 적용시키기 (함수 하나씩만 써도 됨)     
for file_name in jsonl_file:
    convert_rag_dialog(file_name)

In [94]:
convert_rag_dialog('output')

In [72]:
qa_table = pd.read_excel('output.xlsx')

In [76]:
sample_text = qa_table.loc[0,'ori_text']

In [86]:
sample_text = '제1장 총 칙 제1조(목 적) 이 규정은 육군의 인력정책 수립과 인력운영에 필요한 제대별 임무 및 책임의 (한계, 인력관리 업무수행)에 관한 제반 절차를 규정함을 목적으로 한다.'

In [87]:
idx = sample_text.find(')')
doc_idx = sample_text[:idx+1].strip()
doc_cont = sample_text[idx+1:].strip()

In [88]:
doc_idx

'제1장 총 칙 제1조(목 적)'